In [169]:
import os 
import pandas as pd
pd.options.display.max_colwidth = 100
import altair as alt
alt.renderers.enable('html')
alt.data_transformers.disable_max_rows() # altair now handles larger datasets

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [170]:
# Data import

root = "/Volumes/TOB_WD2/Image_Analysis/TPX2/DataFrames" + "/"
df_Path = root + "MasterDataFrame_TPX2.csv"

df = pd.read_csv(df_Path)

print("The shape of the df after import: " + str(df.shape))

The shape of the df after import: (324, 25)


In [174]:
#set filter
df = df[(df.Cell_Volume_um3 < 4500) | (df.Cell_Volume_um3.isna())]
df = df[(df.Spindle_Volume_um3 < 500) | (df.Spindle_Volume_um3.isna())]

Keep_day = ["D0", "D2"] 

df = df[df['Condition'].isin(Keep_day)]

In [177]:
colourscheme = "cividis"

In [178]:
def stripbox(data, x, y, y_title, colour):
    boxplot = alt.Chart().mark_boxplot(
        extent = 'min-max', 
        size = 12
    ).encode(
        y = alt.Y(y, title = y_title),
        opacity = alt.value(1),
        stroke = alt.value('black'),
        color = alt.value('white')
    ).properties(
        width = 30,
        height = 200
    )

    stripplot = alt.Chart().mark_circle(
        size = 20, opacity = 1
    ).encode(
        x = alt.X(
            'jitter:Q',
            title = None,
            axis = alt.Axis(values = [0], grid = False, labels = False, ticks = True),
        ),
        y = alt.Y(y, title = y_title, 
            axis = alt.Axis(grid = False, labels = True, ticks = True)),
        color = alt.Color(colour, scale = alt.Scale(scheme = colourscheme), legend = None)
        ).transform_calculate(
            jitter = '(sqrt(-2 * log(random() / 2)) * cos(2 * PI * random() / 2))'
    ).properties(
        width = 15,
        height = 200
    )
    
    FACETCHART = alt.layer(
        stripplot, boxplot, data = data
        ).facet(
            column = alt.Column(x, header = alt.Header(
                labelAngle = -90,
                titleOrient = 'top',
                labelOrient = 'bottom',
                labelAlign = 'right',
                labelPadding = 5)
                )
        ).configure_facet(
            spacing = 18
        ).configure_axis(
                grid = True, ticks = True, labelPadding = 5
        ).configure_header(
            labelOrient = 'bottom', title = None
        ).configure_view(
            stroke = 'transparent', 
            strokeWidth = 0.5
        )
    return FACETCHART

In [179]:
def Group(dataframe, x, y, y_title, color, column, 
          BoxOpacity = 1, 
          Group_width = 50, 
          Group_height = 200, 
          size = 12):
    
    GROUP = alt.Chart(
        data = dataframe, 
        width = Group_width, 
        height = Group_height
    ).mark_boxplot(
        opacity = BoxOpacity,
        extent = "min-max",
        size = size
    ).encode(
        alt.X(x, title = None),
        alt.Y(y, title = y_title),
        color = alt.Color(color, scale = alt.Scale(scheme = colourscheme)),
        column = alt.Column(column)
    )
    return GROUP

In [180]:
def Scatter_bin(dataframe, x, y, color, x_title, y_title, binextent, binstep,
            Circlesize = 15, 
            CircleOpacity = 1,  
            Scatter_width = 250, 
            Scatter_height = 250
               ):
    # Standard scatter plot 
    SCATTER = alt.Chart(
        data = dataframe, 
        width = Scatter_width, 
        height = Scatter_height
    ).mark_circle(
        opacity = CircleOpacity,
        size = Circlesize
    ).encode(
        alt.X(x, title = x_title),
        alt.Y(y, title = y_title),
        color = alt.Color(
            color, scale = alt.Scale(scheme = colourscheme)#, legend = None#, scale = None
        ) 
    )
    
    SCATTER_bin = alt.Chart(
        data = dataframe, 
        width = Scatter_width, 
        height = Scatter_height
    ).mark_circle(
        opacity = 1,
        size = 150
    ).encode(
        alt.X(x, title = x_title, bin = alt.Bin(extent = binextent, step = binstep)),
        alt.Y("median(" + y + ")", title = y_title, bin = False),
        color = alt.Color(
            color, scale = alt.Scale(scheme = colourscheme)#, legend = None#, scale = None
        ) 
    )
    
    ERROR_Scatterbin = alt.Chart(
            data = dataframe
    ).mark_errorbar(extent = "iqr").encode(
        alt.X(x, title = x_title, bin = alt.Bin(extent = binextent, step = binstep)),
        alt.Y("median(" + y + ")", title = y_title, bin = False),
        color = alt.Color(
            color, scale = alt.Scale(scheme = colourscheme)#, legend = None#, scale = None
        ) 
    ) 

    
    SCATTERBIN = SCATTER + ERROR_Scatterbin + SCATTER_bin 
    return SCATTERBIN

In [194]:
GROUP_TPX2_Spindle_norm = Group(
    dataframe = df, 
    x = "Condition", 
    y = "Average_TPX2_Spindle_norm", 
    y_title = "TPX2 norm. average on spindle",
    color = "Condition", 
    column = "Cells"
)
GROUP_TPX2_Spindle_norm

alt.Chart(...)

In [198]:
STRIP_TPX2_Spindle_percent = stripbox(
    data = df, 
    x = "Condition:N", 
    y = "TPX2_mass_at_Spindle_percent", 
    y_title = "% of total TPX2 mass on spindle",
    colour = "Condition:N"
)
STRIP_TPX2_Spindle_percent

alt.FacetChart(...)

In [200]:
SCATTER_CV_TPX2 = alt.Chart(
        data = df, 
        width = 200, 
        height = 200
    ).mark_circle(
        opacity = 1,
        size = 200
    ).encode(
        alt.X("mean(Cell_Volume_um3)", title = "Cell Volume (µm3"),
        alt.Y("mean(TPX2_mass_Spindle_norm)", title = "TPX2_mass_Spindle_norm"),
        color = alt.Color("Condition", scale = alt.Scale(scheme = colourscheme))
    )
SCATTER_CV_TPX2

alt.Chart(...)

In [201]:
SCATTER_CV_TPX22 = alt.Chart(
        data = df, 
        width = 200, 
        height = 200
    ).mark_circle(
        opacity = 1,
        size = 20
    ).encode(
        alt.X("Cell_Volume_um3", title = "Cell Volume (µm3"),
        alt.Y("TPX2_mass_Spindle_norm", title = "TPX2_mass_Spindle_norm"),
        color = alt.Color("Condition", scale = alt.Scale(scheme = colourscheme))
    )
SCATTER_CV_TPX22

alt.Chart(...)

In [202]:
SCATTERBIN_CV_TPX2 = Scatter_bin(
    dataframe = df, 
    x = "Cell_Volume_um3", 
    x_title = "Cell volume (µm3)",
    y = "TPX2_mass_Spindle_norm",
    y_title = "TPX2 mass on spindle (a.u.)",
    color = "Condition",
    binextent = [0, 4000],
    binstep = 500
   # Circlesize = 5,
   # CircleOpacity = 1
)
SCATTERBIN_CV_TPX2

alt.LayerChart(...)

In [203]:
SCATTER_SV_TPX2mean = alt.Chart(
        data = df, 
        width = 200, 
        height = 200
    ).mark_circle(
        opacity = 1,
        size = 200
    ).encode(
        alt.X("mean(Spindle_Volume_um3)", title = "Spindle Volume (µm3"),
        alt.Y("mean(TPX2_mass_Spindle_norm)", title = "TPX2_mass_Spindle_norm"),
        color = alt.Color("Condition", scale = alt.Scale(scheme = colourscheme))
    )
SCATTER_SV_TPX2mean

alt.Chart(...)

In [204]:
SCATTER_SV_TPX2 = alt.Chart(
        data = df, 
        width = 200, 
        height = 200
    ).mark_circle(
        opacity = 1,
        size = 20
    ).encode(
        alt.X("Spindle_Volume_um3", title = "Spindle Volume (µm3"),
        alt.Y("TPX2_mass_Spindle_norm", title = "TPX2_mass_Spindle_norm"),
        color = alt.Color("Condition", scale = alt.Scale(scheme = colourscheme))
    )
SCATTER_SV_TPX2

alt.Chart(...)

In [205]:
SCATTERBIN_SV_TPX2 = Scatter_bin(
    dataframe = df, 
    x = "Spindle_Volume_um3", 
    x_title = "Spindle volume (µm3)",
    y = "TPX2_mass_Spindle_norm",
    y_title = "TPX2 mass on spindle (a.u.)",
    color = "Condition",
    binextent = [0, 500],
    binstep = 50
   # Circlesize = 5,
   # CircleOpacity = 1
)
SCATTERBIN_SV_TPX2

alt.LayerChart(...)

In [206]:
SCATTERBIN_SV_TPX2conc = Scatter_bin(
    dataframe = df, 
    x = "Spindle_Volume_um3", 
    x_title = "Spindle volume (µm3)",
    y = "Average_TPX2_Spindle_norm",
    y_title = "TPX2 average on spindle (normalised)",
    color = "Condition",
    binextent = [0, 450],
    binstep = 100
   # Circlesize = 5,
   # CircleOpacity = 1
)
SCATTERBIN_SV_TPX2conc

alt.LayerChart(...)

In [209]:
# Export Dataframe

destination = "/Volumes/TOB_WD2/Image_Analysis/TPX2/Dataframes" + "/MasterDataFrame_TPX2_for_stats.csv"
df.to_csv(destination)
print("Successfully saved dataframe at {}".format(destination))

Successfully saved dataframe at /Volumes/TOB_WD2/Image_Analysis/TPX2/Dataframes/MasterDataFrame_TPX2_for_stats.csv
